In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os

from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras import optimizers
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Fixadas considerando as 7 classes da base de dados papsmear
NUM_CLASSES = 7

# Fixada para imagens de três canais RGB
CHANNELS = 3

IMAGE_RESIZE = 200

RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'

OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Métrica de precisão comum para todas as saídas, mas pode usar métricas diferentes para saídas diferentes
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE deve ser < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 10

# O valor dessas etapas deve ser FACTOR adequado de nº de imagens em pastas de treinamento e teste, respectivamente
# As imagens de treinamento processadas em cada etapa seriam no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING   = 8
STEPS_PER_EPOCH_VALIDATION = 2

# O valor dessas etapas deve ser FACTOR adequado de nº de imagens em pastas de treinamento e teste, respectivamente
# NOTE que estes BATCH * são para lotes de Keras ImageDataGenerator para preencher a entrada de etapa de época
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

In [ ]:
resnet_weights_path = 'imagenet'

In [ ]:
# TRANSFER LEARNING

model = Sequential()

#Definindo a 1ª camada (Utilizando a aplicação ResNet50)
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = resnet_weights_path))

# Definindo a 2ª camada como Densa para classificação de 7 classes usando a ativação do SoftMax
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# Diga para não treinar o modelo de primeira camada (ResNet), pois ela já está treinado, isso considerando o 
# conceito de FROZEN no modelo base, ou seja, na primeira camada, esse é o conceito base
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
#Compilando o modelo de transferência de aprendizagem (Otimizador SGD)
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [ ]:
# Data augmentation 
# Observação: O ideal é utilizar o conceito de data augmentation apenas no conjunto de imagens de treino, entretanto 
    # para fins de teste foi utilizado nos dois conjuntos de imagens (treino e teste)

image_size = IMAGE_RESIZE
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
        'images/treino',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        'images/validacao',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical')

In [ ]:
# Número máximo de etapas que esses geradores terão oportunidade de processar seu conteúdo de origem
# len (train_generator) deve ser 'nº de imagens de trem disponíveis / BATCH_SIZE_TRAINING '
# len (valid_generator) deve ser 'nº de imagens de trem disponíveis / BATCH_SIZE_VALIDATION '

(BATCH_SIZE_TRAINING, len(train_generator), BATCH_SIZE_VALIDATION, len(validation_generator))
#(BATCH_SIZE_TRAINING, len(train_generator), BATCH_SIZE_VALIDATION)

In [ ]:
# EarlyStopping => Pare o treinamento quando uma métrica monitorada parar de melhorar.
# Assumindo que o objetivo de um treinamento é minimizar a perda. Com isso, a métrica a ser monitorada seria 'loss',
#   e o modo seria 'min'. Um model.fit()loop de treinamento verificará ao final de cada época se a perda não está mais 
#   diminuindo, considerando o min_deltae patiencese aplicável. Uma vez que não esteja mais diminuindo, model.stop_trainingé
#   marcado como Verdadeiro e o treinamento termina.

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)

# ModelCheckpoint => Retorno de chamada para salvar o modelo Keras ou pesos do modelo com alguma frequência.
# ModelCheckpointO retorno de chamada é usado em conjunto com o treinamento model.fit()
#  para salvar um modelo ou pesos (em um arquivo de ponto de verificação) em algum intervalo, 
#  para que o modelo ou pesos possam ser carregados posteriormente para continuar o treinamento do estado salvo.

cb_checkpointer = ModelCheckpoint(filepath = 'images/rede.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [ ]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
    epochs = NUM_EPOCHS,
    validation_data=validation_generator,
    validation_steps=STEPS_PER_EPOCH_VALIDATION,
    callbacks=[cb_checkpointer, cb_early_stopper]
)

# LOAD_WEIGHTS => O savefile inclui:
# A arquitetura do modelo, permitindo reinstanciar o modelo.
# Os pesos do modelo.
# O estado do otimizador, permitindo retomar o treinamento exatamente de onde você parou.
# Isso permite que você salve todo o estado de um modelo em um único arquivo.

In [ ]:
# Métricas de treinamento
print(fit_history.history.keys())

In [ ]:
plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(fit_history.history['accuracy'])  
plt.plot(fit_history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid'])
    
plt.subplot(222)  
plt.plot(fit_history.history['loss'])  
plt.plot(fit_history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'])

plt.show()